# TKO_7093 - Statistical Data Analysis Project
## Group 160

## Team Members

| Name                      | Student ID|
| :--------                 | :-------: |
| Ayana Kotuwegoda Guruge   | 2406865   |
| Sheheryar Wahidi          | 2413773   |
| Yagya Yadav               | 2409273   |

---

## Data Preparation

In [10]:
#Needed libraries
#from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [11]:
#drive.mount('/content/drive')
#!ls /content/drive/


### Following steps are carried out for the data preparation:

This step is for checking the dataset, cleaning and structuring for further analysis (qualititative). 

* Data loaded from habits.data into a DataFrame matching the variables.
* Missing values are checked and not removed.
* Codes are converted into readable lables.
* Checking all demographic values for sensible values.
* Checking the validity of the ranges.
* Replacing the invalid values of the variables accordingly.

In [12]:
# Load the dataset from Path
data_path = "habits.data"
columns = ['kohde', 'jasen', 'pvknro', 'sp', 'ASALUE', 'IKAL1', 'A1', 'A2', 'A3', 'A4', 'A5']
df = pd.read_csv(data_path, sep=";", header=0, usecols=columns, na_values=['?'])

# Initial exploration of the dataset    
df.head()
df.shape
df.info()
df.describe()

# Display the first few rows of the dataframe
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kohde   745 non-null    int64  
 1   jasen   745 non-null    int64  
 2   pvknro  745 non-null    int64  
 3   sp      745 non-null    int64  
 4   ASALUE  745 non-null    float64
 5   IKAL1   745 non-null    int64  
 6   A1      741 non-null    object 
 7   A2      737 non-null    object 
 8   A3      733 non-null    object 
 9   A4      736 non-null    object 
 10  A5      703 non-null    float64
dtypes: float64(2), int64(5), object(4)
memory usage: 64.2+ KB


,kohde,jasen,pvknro,sp,ASALUE,IKAL1,A1,A2,A3,A4,A5
0,50002,1,1,1,1.0,49,0,560,0,80,1.0
1,50002,1,2,1,1.0,49,380,450,10,0,1.0
2,50003,1,1,2,2.0,41,0,470,30,100,1.0
3,50003,1,2,2,2.0,41,0,550,0,0,1.0
4,50004,2,1,1,1.0,62,640,410,0,0,1.0


* Structure was checked because according to the instructions given as there are multiple rows of the same person.
* This is more to show that one person can be recorded for more than one day. Changes will be done in task 1
    * 6 columns (Household ID, Person ID, day type, sex, living area, age group) are demographic variables.
    * 5 columns (A1-A5) which are activity variables.
* A1 to A4 variables are shown as objects where it should be in numerics since it is the time spent in activities in minutes according to the text file. This is due to mixed formatting or missing values or both.
* The appearance of A5 variable (Visiting library) in float which aslo should be in numerics.

In [14]:
# Checking for missing values
print("Missing values in each column:")
print(df.isnull().sum())

Missing values in each column:
kohde      0
jasen      0
pvknro     0
sp         0
ASALUE     0
IKAL1      0
A1         4
A2         8
A3        12
A4         9
A5        42
dtype: int64


* It shows that there some missing values in the activity variables but no missing values in the demographic variables.

* Number of missing values of activity columns:
    * A1 - 4
    * A2 - 8
    * A3 - 12
    * A4 - 9
    * A5 - 42 

* Missing values are kept as NaN.
* Missing values are not always "0 minutes spent"/" no activity". It can also mean that the data entry was missed to be recorded or an error. Therefore, it is better to keep them as NaN value guessing or maintaining as 0 assuming that there was no activity.

### Data Type Conversion and Cleaning

In [15]:
# Replace '?' with NaN for consistent value handling
df_clean = df.replace('?', np.nan)

# Convert demographic variables to appropriate types respectrively (all numeric in this case)
df_clean['kohde'] = pd.to_numeric(df_clean['kohde'], errors='coerce')
df_clean['jasen'] = pd.to_numeric(df_clean['jasen'], errors='coerce')
df_clean['pvknro'] = pd.to_numeric(df_clean['pvknro'], errors='coerce')
df_clean['sp'] = pd.to_numeric(df_clean['sp'], errors='coerce')
df_clean['IKAL1'] = pd.to_numeric(df_clean['IKAL1'], errors='coerce')
df_clean['ASALUE'] = pd.to_numeric(df_clean['ASALUE'], errors='coerce')

# Activity variables converting to numeric, handling time format data
for col in ['A1', 'A2', 'A3', 'A4', 'A5']:
    # A1-A4 are minutes and A5 in yes/no format, convert accordingly
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print("Data types after conversion:")
print(df_clean.dtypes)
print("\nMissing values per column")
print(df_clean.isnull().sum())
print("\nDataFrame shape:", df_clean.shape)

Data types after conversion:
kohde       int64
jasen       int64
pvknro      int64
sp          int64
ASALUE    float64
IKAL1       int64
A1        float64
A2        float64
A3        float64
A4        float64
A5        float64
dtype: object

Missing values per column
kohde      0
jasen      0
pvknro     0
sp         0
ASALUE     0
IKAL1      0
A1        85
A2        88
A3        94
A4        91
A5        42
dtype: int64

DataFrame shape: (745, 11)


The above conversions will ensure that later calculations will be correct since all the incorrect and missing values are now treated as missing values (NaN). This was done using 'errors="coerce"' where cleaning up will not create new missing data but changing the already existing ones.

In [16]:
# Checking for ranges from the text file for variables pvknro, sp, ASALUE, IKAL1
allowed_ranges = {
    'pvknro' : {1,2},
    'sp' : {1,2},
    'ASALUE' : {1,2,3},
    'IKAL1' : set(range(1,10)), # 1 to 9 
    'A5' : {1,2} # yes/no  
}

for col, valid_values in allowed_ranges.items():
    unique_values = set(df[col].dropna().unique())
    invalid_values = unique_values - valid_values
    print(f"{col}:")
    print(f"Unique values found: {sorted(unique_values)}")
    if invalid_values:
        print(f"Invalid values found: {sorted(invalid_values)}")
    else:
        print("All values are valid.")
    print()

pvknro:
Unique values found: [np.int64(1), np.int64(2)]
All values are valid.

sp:
Unique values found: [np.int64(1), np.int64(2)]
All values are valid.

ASALUE:
Unique values found: [np.float64(1.0), np.float64(2.0), np.float64(3.0)]
All values are valid.

IKAL1:
Unique values found: [np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), np.int64(69), np.int64(70),

* Above verification is done because the missing values per column increased after converting the data types for the correct ones. 
    * pvknro, sp, ASALUE are all valid.
    * IKAL1 which are age ranges which should only be from 1-9 have values more than 20. 
        * Therefore, it is assumed that the ages are entered insted of the relevant age group number.
    * A5 should be having yes or no entries with either 1 or 2 to depict that the library was visited or not. But there are some float values entered. 
        * Therefore, it is assumed that those float values are the number of hours spent in the library which will then be entered as 1 (yes) to which it will be changed.

#### Data correction